In [2]:
import os
import memoized # poczytaj co to sa dekoratory
import pandas as pd
from Bio.KEGG import REST


### Get genomes

In [3]:
@memoized.cache # to jest specjalny dekorator funkcji, ktory dziala jak cache wynikow
                # jak cos raz policzysz, to potem wynik jest naczytywany z pliku ./cache/xxx
def get_organisms():
    organisms = REST.kegg_list('organism').read()
    organisms = [i.split('\t') for i in organisms.split('\n')]
    return(organisms)

In [4]:
organisms = get_organisms()
prokaryotes_codes = [organism[1] for organism in organisms if 'Prokaryotes' in organism[-1]]

### Get pathways

In [5]:
@memoized.cache
def get_pathways():
    pathways = REST.kegg_list('pathway').read()
    pathways = [i.split('\t') for i in pathways.split('\n')]
    return pathways

In [6]:
pathways = get_pathways()
pathways = pathways[:1] # !!! we take just one for testing purposes !!!

### Generate data

In [7]:
@memoized.cache
def get_kos(pathway):
    kos = REST.kegg_link('ko', pathway).read()
    kos = [i.split('\t') for i in kos.split('\n')]
    return kos

In [8]:
@memoized.cache    
def get_genes(ko):
    genes = REST.kegg_link('genes', ko).read()
    genes = [i.split('\t') for i in genes.split('\n')]
    return genes

In [9]:
def fix(i):
    assert isinstance(i, list)
    if i[-1] == ['']: 
        return i[:-1]
    else:
        return i

In [10]:
path2ko_and_genes = {}

for path_id, path_name in pathways:
    """
     map Reference pathway
     ko  Reference pathway (KO only)
     ec  Reference pathway (EC only)
     rn  Reference pathway (Reaction only)
    """
    path_id = path_id.replace('path:map', 'path:ko')
    kos = get_kos(path_id)
    kos = fix(kos)
    
    print(path_id, 'contains', len(kos), 'KO')
    
    ko2genes = {}
    
    for _, ko in kos:
        print('\t', ko)
        genes = get_genes(ko)
        genes = fix(genes)
        genes = [i[1].split(':') for i in genes]
        
        genes_df = pd.DataFrame(genes, columns=['genome', 'gene_id'])
        genes_df = genes_df[genes_df.genome.apply(lambda x:x in prokaryotes_codes)]
        org2genes = dict([(g[0], g[1].gene_id.tolist()) for g in genes_df.groupby('genome')])
        
        assert not ko in ko2genes
        ko2genes[ko] = org2genes
        
    path2ko_and_genes[path_id] = pd.DataFrame.from_dict(ko2genes)

print('done!')


path:ko00010 contains 106 KO
	 ko:K00001
	 ko:K00002
	 ko:K00016
	 ko:K00114
	 ko:K00121
	 ko:K00128
	 ko:K00129
	 ko:K00131
	 ko:K00134
	 ko:K00138
	 ko:K00149
	 ko:K00150
	 ko:K00161
	 ko:K00162
	 ko:K00163
	 ko:K00169
	 ko:K00170
	 ko:K00171
	 ko:K00172
	 ko:K00174
	 ko:K00175
	 ko:K00189
	 ko:K00382
	 ko:K00627
	 ko:K00844
	 ko:K00845
	 ko:K00850
	 ko:K00873
	 ko:K00886
	 ko:K00895
	 ko:K00918
	 ko:K00927
	 ko:K01006
	 ko:K01007
	 ko:K01084
	 ko:K01085
	 ko:K01086
	 ko:K01222
	 ko:K01223
	 ko:K01568
	 ko:K01596
	 ko:K01610
	 ko:K01622
	 ko:K01623
	 ko:K01624
	 ko:K01689
	 ko:K01785
	 ko:K01792
	 ko:K01803
	 ko:K01810
	 ko:K01834
	 ko:K01835
	 ko:K01837
	 ko:K01895
	 ko:K01905
	 ko:K01913
	 ko:K02446
	 ko:K02753
	 ko:K02777
	 ko:K02779
	 ko:K02791
	 ko:K03103
	 ko:K03737
	 ko:K03841
	 ko:K04022
	 ko:K04041
	 ko:K04072
	 ko:K06859
	 ko:K08074
	 ko:K10705
	 ko:K11389
	 ko:K11532
	 ko:K11645
	 ko:K12406
	 ko:K12407
	 ko:K12957
	 ko:K13810
	 ko:K13951
	 ko:K13952
	 ko:K13953
	 ko:K13954

In [11]:
# path2ko_and_genes to jest dict sciezka -> df, np.:

path2ko_and_genes['path:ko00010']

,ko:K00001,ko:K00002,ko:K00016,ko:K00114,ko:K00121,ko:K00128,ko:K00129,ko:K00131,ko:K00134,ko:K00138,...,ko:K18978,ko:K20118,ko:K20866,ko:K21071,ko:K22224,ko:K22473,ko:K22474,ko:K24012,ko:K24182,ko:K25026
aaa,NaN,NaN,NaN,NaN,[Acav_2947],"[Acav_1053, Acav_2451, Acav_2537]",NaN,NaN,[Acav_4518],[Acav_2020],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Acav_3099],NaN,NaN
aab,NaN,NaN,NaN,NaN,NaN,"[A4R43_30060, A4R43_34840, A4R43_07835, A4R43_...",NaN,NaN,[A4R43_04915],[A4R43_24185],...,NaN,NaN,NaN,[A4R43_16320],NaN,NaN,NaN,[A4R43_27965],NaN,"[A4R43_26355, A4R43_16360]"
aac,[Aaci_0920],NaN,[Aaci_0520],NaN,NaN,"[Aaci_1057, Aaci_0749, Aaci_0152]",NaN,NaN,[Aaci_2273],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Aaci_0886],NaN,[Aaci_0086]
aace,NaN,NaN,NaN,[A0U92_15830],[A0U92_04530],NaN,NaN,NaN,[A0U92_02035],[A0U92_12420],...,NaN,NaN,NaN,NaN,NaN,[A0U92_12090],"[A0U92_12095, A0U92_13970, A0U92_15155]",NaN,NaN,NaN
aaci,NaN,NaN,"[ASQ49_04850, ASQ49_04870]",NaN,NaN,NaN,NaN,NaN,"[ASQ49_16350, ASQ49_14600]",[ASQ49_03595],...,NaN,NaN,NaN,[ASQ49_03930],NaN,NaN,NaN,NaN,NaN,[ASQ49_14985]
aacn,[AANUM_0982],NaN,NaN,NaN,[AANUM_0855],NaN,NaN,NaN,[AANUM_0173],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aact,NaN,NaN,NaN,NaN,[ACT75_05650],NaN,NaN,NaN,[ACT75_07580],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aad,NaN,NaN,NaN,NaN,NaN,"[TC41_0193, TC41_0723, TC41_1093]",NaN,NaN,[TC41_2560],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[TC41_0894],NaN,[TC41_0106]
aae,[aq_1240],NaN,NaN,NaN,NaN,NaN,NaN,NaN,[aq_1065],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aagg,NaN,NaN,"[ETAA8_30760, ETAA8_50800]",[ETAA8_14430],NaN,"[ETAA8_30520, ETAA8_58900]",NaN,NaN,[ETAA8_38110],NaN,...,NaN,NaN,NaN,"[ETAA8_03950, ETAA8_18410, ETAA8_54750, ETAA8_...",NaN,NaN,NaN,NaN,NaN,"[ETAA8_38370, ETAA8_59170, ETAA8_44420]"


In [14]:
path2ko_and_genes['path:ko00010'].columns

Index(['ko:K00001', 'ko:K00002', 'ko:K00016', 'ko:K00114', 'ko:K00121',
       'ko:K00128', 'ko:K00129', 'ko:K00131', 'ko:K00134', 'ko:K00138',
       ...
       'ko:K18978', 'ko:K20118', 'ko:K20866', 'ko:K21071', 'ko:K22224',
       'ko:K22473', 'ko:K22474', 'ko:K24012', 'ko:K24182', 'ko:K25026'],
      dtype='object', length=106)

TO DO:

1. napisz funkcje `cleandf(df, max_nan_fraction)`, ktora usuwa z zadanego df wszystkie kolumny, ktore maja wiecej niz x% NaN (jest sporo KO, dla ktorych sa geny tylko dla niewielkiej czesci genomow). zbadaj funkcje `df.drop` i jej parametr `inplace=True`

2. napisz funkcje `getseq(gene)`, ktora dla zadanego genu (np. Acav_1053, ASQ49_04850m itp.) zwroci jego sekwencje bialkowa

3. zainstaluj sobie paczke `rossmann-toolbox` (https://github.com/labstructbioinf/) i skonfiguruj tak, zeby Ci dzialala podstawowa funkcjonalnosc:

```
from rossmann_toolbox import RossmannToolbox
rtb = RossmannToolbox(use_gpu=True)

data = {'3m6i_A': 'MASSASKTNIGVFTNPQHDLWISEASPSLESVQKGEELKEGEVTVAVRSTGICGSDVHFWKHGCIGPMIVECDHVLGHESAGEVIAVHPSVKSIKVGDRVAIEPQVICNACEPCLTGRYNGCERVDFLSTPPVPGLLRRYVNHPAVWCHKIGNMSYENGAMLEPLSVALAGLQRAGVRLGDPVLICGAGPIGLITMLCAKAAGACPLVITDIDEGRLKFAKEICPEVVTHKVERLSAEESAKKIVESFGGIEPAVALECTGVESSIAAAIWAVKFGGKVFVIGVGKNEIQIPFMRASVREVDLQFQYRYCNTWPRAIRLVENGLVDLTRLVTHRFPLEDALKAFETASDPKTGAIKVQIQSLE'}

preds = rtb.predict(data, mode='seq')
```